# Chapter 15 - Google Sheets

## Notes

### Installing and Setting Up EZSheets
You can install EZSheets with the pip command line tool by following the instructions in Appendix A.

Before your Python scripts can use EZSheets to access and edit your Google Sheets spreadsheets, you need a credentials JSON file and two token JSON files. There are five parts to creating credentials:

  1.  Create a new Google Cloud project.

  2.  Enable the Google Sheets API and Google Drive API for your project.

  3.  Configure the OAuth consent screen.

  4.  Create credentials.

  5.  Log in with the credentials file.

### Project 11: Fake Blockchain Cryptocurrency Scam
In this project, we’ll use Google Sheets as a fake blockchain to track the transactions of Boringcoin, a cryptocurrency scam I’m promoting. (It turns out that investors and customers don’t care if your blockchain product uses a real blockchain data structure; they will give you money anyway.)

The URL https://autbor.com/boringcoin redirects to the Google Sheets URL for Boringcoin’s blockchain. The spreadsheet has three columns: the sender of the transaction, the recipient of the transaction, and the amount of the transaction. The amount is deducted from the sender and added to the recipient. If the sender is 'PRE-MINE', this money is created out of thin air and added to the recipient account. Figure 15-5 shows this Google Sheet.

The first transaction has the sender 'PRE-MINE' and the recipient 'Al Sweigart', and the amount is a humble 1000000000. The 'Al Sweigart' account then transfers 19116 Boringcoins to 'Miles Bron', who then transfers 118 Boringcoins to 'not_a_scammer'. The fourth transaction transfers 16273 Boringcoins from 'Al Sweigart' to 'some hacker'. (I did not authorize this transaction and have since stopped using python12345 as my Google account password.)

Let’s write two programs. First, the auditBoringcoin.py program examines all the transactions and generates a dictionary of all accounts and their current balance. Second, the addBoringcoinTransaction.py program adds a row to the end of the Google Sheets for a new transaction. These blockchain programs are just for fun and not real (though “real” blockchain projects such as NFTs and “web3” are just as much a fantasy).

#### Step 1: Audit the Fake Blockchain
We need to write a program to examine the entire “blockchain” and determine the current balance of all accounts. We’ll use a dictionary to hold this data, where the keys are strings of the account name and the values are integers of how many Boringcoins are in them. We also want the program to display how many total Boringcoins are in the cryptocurrency network. We can start by importing EZSheets and setting up the dictionary:
```
import ezsheets
ss = ezsheets.Spreadsheet('https://autbor.com/boringcoin')
accounts = {}  # Keys are names, and values are amounts.
```
Next, we’ll loop through every row in the spreadsheet, identifying the sender, recipient, and amount. Keep in mind that Google Sheets always returns data as a string, so we need to convert it to an integer to do math with the amount value:
```
# Each row is a transaction. Loop over each one:
for row in ss.sheets[0].getRows():
    sender, recipient, amount = row[0], row[1], int(row[2])
```
If the sender is the special account 'PRE-MINE', then it is simply a source of infinite money into other accounts. All of the best cryptocurrency scams use pre-mined coins, and ours is no exception. Add the amount to the recipient account in the accounts dictionary. The setdefault() method sets the value of the account to 0 if it doesn’t already exist in the dictionary:
```
    if sender == 'PRE-MINE':
        # The 'PRE-MINE' sender invents money out of thin air.
        accounts.setdefault(recipient, 0)
        accounts[recipient] += amount
```
Otherwise, we should deduct the amount from the sender and add it to the recipient:
```
    else:
        # Move funds from the sender to the recipient.
        accounts.setdefault(sender, 0)
        accounts.setdefault(recipient, 0)
        accounts[sender] -= amount
        accounts[recipient] += amount
```
After the loop finishes, we can see the current balances by printing the accounts dictionary.

`print(accounts)`
As part of our audit, let’s also go through this dictionary and add up the totals of everyone’s balance to find out how many Boringcoins are in the entire network. Start a total variable at 0, and then have a for loop go through each value in the key-value pairs of the accounts dictionary. After adding each value to total, we can print the total amount of Boringcoins:
```
total = 0
for amount in accounts.values():
    total += amount
print('Total Boringcoins:', total)
```
When we run this program, the output looks like this:
```
{'Al Sweigart': 999058553, 'Miles Bron': 38283, 'not_a_scammer': 48441,
'some hacker': 44429, 'Tech Bro': 53424, 'Claire Debella': 54443,
'Credulous Journalist': 50408, 'Birdie Jay': 36832, 'Carol': 82867, 'Mark Z.':
 68650, 'Bob': 37920, 'Andi Brand': 57218, 'Eve': 88296, 'Al Sweigart sock
#27': 78080, 'Tax evader': 40937, 'Duke Cody': 17544, 'Lionel Toussaint':
54650, 'some scammer': 2694, 'Alice': 44503, 'David': 41828}
Total Boringcoins: 1000000000
```
The total is 1000000000, which makes sense, because that’s how many Boringcoins were pre-mined.

#### Step 2: Make Transactions
The next program, addBoringcoinTransaction.py, adds additional rows to the “blockchain” Google Sheet to add new transactions. It reads three command line arguments from the list in sys.argv: the sender, the recipient, and the amount. For example, you could run the following from the terminal:

`python addBoringcoinTransaction.py "Al Sweigart" Eve 2000`
The program would access the Google Sheet, add a blank row to the bottom, and then fill it in with the values 'Al Sweigart', 'Eve', and '2000'. Note that in the terminal, you’ll need to enclose any command line argument that contains a space in double quotes, like "Al Sweigart"; otherwise, the terminal will think they are two separate arguments.

The start of addBoringcoinTransactions.py checks the command line arguments and assigns the sender, recipient, and amount variables based on them:
```
import sys, ezsheets

if len(sys.argv) < 4:
    print('Usage: python addBoringcoinTransaction.py sender recipient amount')
    sys.exit()

# Get the transaction info from the command line arguments:
sender, recipient, amount = sys.argv[1:]
```
You won’t need to convert amount from a string to an integer, because we’ll be writing it as a string to the spreadsheet.

Next, EZSheets connects to the Google Sheets containing the fake blockchain and selects the first sheet in the spreadsheet (at index 0). Note that you don’t have permission to edit the Boringcoin Google Sheets, so open that URL in a web browser while logged in to your Google account and then select FileMake a Copy to copy it to your Google Account. Then, replace the 'https://autbor.com/boringcoin' string with a string of your Google Sheet’s URL from the browser address bar:
```
# Change this URL to your copy of the Google Sheet, or else you'll
# get a "The caller does not have permission" error.
ss = ezsheets.Spreadsheet('https://autbor.com/boringcoin')
sheet = ss.sheets[0]
```
Finally, you should get the number of rows in the sheet, increase it by one, and then fill in the columns of this row with the sender, recipient, and amount data:
```
# Add one more row to the sheet for a new transaction:
sheet.rowCount += 1

sheet[1, sheet.rowCount] = sender
sheet[2, sheet.rowCount] = recipient
sheet[3, sheet.rowCount] = amount
```
Now when you run python addBoringcoinTransaction.py "Al Sweigart" Eve 2000 from the terminal, the Google Sheets will have a new row with Al Sweigart, Eve, and 2000 added at the bottom. You can rerun the auditBoringcoin.py program to see the updated account balances of everyone in the cryptocurrency network.

The use of Google Sheets for our blockchain data structure is irresponsible, error prone, and a security catastrophe waiting to happen. This makes it on par with most marketed blockchain products. Don’t miss out! Contact me to get in on this limited offer to buy Boringcoin before the pyramid scheme collapses!

## Practice Questions

  1.  What three files do you need for EZSheets to access Google Sheets?

  2.  What two types of objects does EZSheets have?

  3.  How can you create an Excel file from a Google Sheets spreadsheet?

  4.  How can you create a Google Sheets spreadsheet from an Excel file?

  5.  The ss variable contains a Spreadsheet object. What code will read data from the cell B2 in a sheet titled Students?

  6.  How can you find the column letters for column 999?

  7.  How can you find out how many rows and columns a sheet has?

  8.  How do you delete a spreadsheet? Is this deletion permanent?

  9.  What functions will create a new Spreadsheet object and a new Sheet object, respectively?

10.  What would happen if, by making frequent read and write requests with EZSheets, you exceed your Google account’s quota?

1. What three files do you need for EZSheets to access Google Sheets?  
    **Answer:** `credentials-sheets.json`, `token.json`, `authorized_user.json`

2. What two types of objects does EZSheets have?  
    **Answer:** `Spreadsheet`, `Sheet`

3. How can you create an Excel file from a Google Sheets spreadsheet?  
    **Answer:** `ss.downloadAsExcel()`

4. How can you create a Google Sheets spreadsheet from an Excel file?  
    **Answer:** `ss = ezsheets.upload('my_spreadsheet.xlsx')`

5. The ss variable contains a Spreadsheet object. What code will read data from the cell B2 in a sheet titled Students?  
    **Answer:** `ss['Students']['B2']`

6. How can you find the column letters for column 999?  
    **Answer:** `ezsheets.getColumnLetterOf(999)`

7. How can you find out how many rows and columns a sheet has?  
    **Answer:** `sheet.rowCount`, `sheet.columnCount`

8. How do you delete a spreadsheet? Is this deletion permanent?  
    **Answer:** `ss.delete()`, no it moves it to the *Trash* folder.

9. What functions will create a new Spreadsheet object and a new Sheet object, respectively?  
    **Answer:** `ss = ezsheets.Spreadsheet()`, `sheet2 = ss.Sheet('Spam')`

10. What would happen if, by making frequent read and write requests with EZSheets, you exceed your Google account’s quota?  
    **Answer:** Attempting to exceed this quota will raise the `googleapiclient.errors.HttpError` “Quota exceeded for quota group” exception.

## Practice Programs

For practice, write programs to do the following tasks.

### Downloading Google Forms Data
I mentioned earlier that Google Forms allows you to create simple online forms that make it easy to collect information from people. The information entered into a form is stored in a Google Sheets spreadsheet. For this project, write a program that can automatically download the form information that users have submitted. Go to https://docs.google.com/forms/ and start a new blank form. Add fields to the form that ask the user for a name and email address. Then, click the Send button in the upper right to get a link to your new form. Try to enter a few example responses into this form.

On the Responses tab of your form, click the green Create Spreadsheet button to create a Google Sheets spreadsheet that will hold the responses that users submit. You should see your example responses in the first rows of this spreadsheet. Then, write a Python script using EZSheets to collect a list of the email addresses on this spreadsheet.

**"download_gform_data.py"**

### Converting Spreadsheets to Other Formats
You can use Google Sheets to convert a spreadsheet file to other formats. Write a script that passes a submitted file to upload(). Once the spreadsheet has uploaded to Google Sheets, download it using downloadAsExcel(), downloadAsODS(), and other such functions to create a copy of the spreadsheet in these other formats.

**"convert_sheets.py"**

### Finding Mistakes in a Spreadsheet
After a long day at the bean-counting office, I’ve finished a spreadsheet with all the bean totals and uploaded them to Google Sheets. The spreadsheet is publicly viewable (but not editable). You can get this spreadsheet with the following code:
```
>>> import ezsheets
>>> ss = ezsheets.Spreadsheet('1jDZEdvSIh4TmZxccyy0ZXrH-ELlrwq8_YYiZrEOB4jg')
```
View the spreadsheet in your browser by going to https://docs.google.com/spreadsheets/d/1jDZEdvSIh4TmZxccyy0ZXrH-ELlrwq8_YYiZrEOB4jg. The columns of the first sheet in this spreadsheet are BEANS PER JAR, JARS, and TOTAL BEANS. The TOTAL BEANS column is the product of the numbers in the BEANS PER JAR and JARS columns. However, there is a mistake in one of the 15,000 rows in this sheet. That’s too many rows to check by hand. Luckily, you can write a script that checks the totals.

As a hint, you can access the individual cells in a row with ss.sheets[0].getRow(rowNum), where ss is the Spreadsheet object and rowNum is the row number. Remember that row numbers in Google Sheets begin at 1, not 0. The cell values will be strings, so you’ll need to convert them to integers before your program can work with them. The expression int(ss.sheets[0].getRow(2)[0]) * int(ss.sheets[0].getRow(2)[1]) == int(ss.sheets[0].getRow(2)[2]) evaluates to True if row 2 has the correct total. Put this code in a loop to identify which row in the sheet has the incorrect total.

**"find_mistake.py"**